# Import

In [1]:
import csv
import os

import numpy as np
import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from tqdm import tqdm

In [2]:
tqdm.pandas()

# Read data

In [3]:
ag_news_df = pd.read_csv("../../DataShaping/data/AgNews/master.csv", index_col=0)

In [4]:
with open("../../DataShaping/data/AgNews/class.csv", mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

# Word tokinize

In [5]:
ag_news_df["words"] = ag_news_df.text.progress_apply(word_tokenize)

100%|██████████| 120000/120000 [00:28<00:00, 4158.56it/s]


In [6]:
ag_news_df["words_nonstop"] = ag_news_df.words.progress_apply(
    lambda words: [
        word for word in words if word.lower() not in stopwords.words("english")
    ]
)

100%|██████████| 120000/120000 [07:10<00:00, 279.06it/s]


In [7]:
ag_news_df.words = ag_news_df.words.progress_apply(lambda words: " ".join(words))
ag_news_df.words_nonstop = ag_news_df.words_nonstop.progress_apply(
    lambda words: " ".join(words)
)

100%|██████████| 120000/120000 [00:00<00:00, 496381.53it/s]


In [8]:
file_path = "../data/AgNews"
os.makedirs(file_path, exist_ok=True)
with open(f"{file_path}/class.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(class_labels)
ag_news_df.to_csv(f"{file_path}/master.csv")